### EXTRAER TRABAJOS DE COMPUTRABAJO APLICANDO TECNICAS DE WEBSCRAPING

In [14]:
!pip install webdriver_manager

  Obtaining dependency information for webdriver_manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
import time
from selenium.webdriver.common.by import By
from selenium import webdriver


from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [21]:
def iniciar_busqueda(departamento, url='https://pe.computrabajo.com/'):
    """
    Inicia sesión en la página de ComputraBajo y realiza una búsqueda inicial por departamento.
    
    :param departamento: Nombre del departamento para la búsqueda
    :type departamento: str
    :param url: URL del sitio de ComputraBajo (opcional)
    :type url: str
    :return: None
    :rtype: None
    """
    driver.get(url)
    time.sleep(5)
    
    # Buscar el departamento
    search_box = driver.find_element(By.ID, 'place-search-input')
    search_box.send_keys(departamento)
    
    # Hacer clic en el botón de búsqueda
    search_button = driver.find_element(By.ID, 'search-button')
    search_button.click()
    
    # Tiempo de espera para que cargue la página de resultados
    time.sleep(3)
    
    return None

In [2]:
def obtener_ultima_url(log_file='ultimo_scraping.txt'):
    """
    Obtiene la última URL procesada del archivo de log.
    
    :param log_file: Ruta al archivo de log
    :type log_file: str
    :return: La última URL procesada o None si no existe
    :rtype: str or None
    """
    try:
        with open(log_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if lines:
                return lines[-1].strip()
    except FileNotFoundError:
        print(f"No se encontró el archivo de log {log_file}. Se iniciará desde el principio.")
    return None

In [3]:
def guardar_ultima_url(url, log_file='ultimo_scraping.txt'):
    """
    Guarda la URL actual en el archivo de log.
    
    :param url: URL a guardar
    :type url: str
    :param log_file: Ruta al archivo de log
    :type log_file: str
    :return: None
    :rtype: None
    """
    with open(log_file, 'a', encoding='utf-8') as file:
        file.write(f"{url}\n")

In [4]:
def main():
    # Configurar el webdriver
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    # chrome_options.add_argument("--headless")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Archivos para almacenar datos y logs
    output_file = 'job_details.json'
    log_file = 'ultimo_scraping.txt'
    
    # Cargar datos previos del JSON
    job_details = {}
    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            content = file.read().strip()
            if content:
                job_details = json.loads(content)
                print(f"Se cargaron {len(job_details)} registros previos del archivo JSON.")
            else:
                print("El archivo JSON existe pero está vacío. Se inicializará con un diccionario vacío.")
    except FileNotFoundError:
        print("No se encontró un archivo previo. Se creará uno nuevo.")
    
    # Obtener la última URL procesada o iniciar búsqueda si no hay
    ultima_url = obtener_ultima_url(log_file)
    
    if ultima_url:
        print(f"Continuando desde la última URL procesada: {ultima_url}")
        driver.get(ultima_url)
        time.sleep(5)
    else:
        print("Iniciando nueva búsqueda desde el principio")
        iniciar_busqueda('Lima')
    
    while True:
        current_url = driver.current_url
        print(f"Procesando página: {current_url}")
        
        try:
            try:
                popup = driver.find_element(By.ID, 'pop-up-webpush-sub')
                close_button = popup.find_element(By.TAG_NAME, 'button')
                close_button.click()
                print("Pop-up cerrado.")
            except:
                print("No se encontró el pop-up.")
            
            # Extraer ofertas de trabajo
            job_offers = driver.find_elements(By.CLASS_NAME, 'box_offer')
            
            print(f"Encontradas {len(job_offers)} ofertas en esta página")
            
            for offer in job_offers:
                try:
                    title_element = offer.find_element(By.CLASS_NAME, 'js-o-link')
                    job_title = title_element.text
                    
                    if job_title in job_details:
                        print(f"Oferta ya procesada: {job_title}")
                        continue
                    
                    offer.click()
                    time.sleep(2)
                    
                    detail_element = driver.find_element(By.CLASS_NAME, 'box_detail')
                    job_details[job_title] = detail_element.get_attribute('innerHTML')
                    
                    print(f"Oferta extraída: {job_title}")
                    
                    driver.back()
                    time.sleep(2)
                except Exception as e:
                    print(f"Error al procesar una oferta: {e}")
            
            with open(output_file, 'w', encoding='utf-8') as file:
                json.dump(job_details, file, ensure_ascii=False, indent=4)
            
            print(f"Total de ofertas guardadas: {len(job_details)}")
            
            guardar_ultima_url(current_url, log_file)
            
            next_button = driver.find_element(By.CLASS_NAME, 'b_primary.w48.buildLink.cp')
            
            if next_button:
                next_url = next_button.get_attribute('href')
                print(f"Pasando a la siguiente página: {next_url}")
                next_button.click()
                time.sleep(5)
            else:
                print("No hay más páginas.")
                break
        
        except Exception as e:
            print(f"Error general en la página {current_url}: {e}")
            # Guardar progreso antes de salir debido a un error
            with open(output_file, 'w', encoding='utf-8') as file:
                json.dump(job_details, file, ensure_ascii=False, indent=4)
            print(f"Se guardaron los datos antes de finalizar por error. Total: {len(job_details)}")
            break
    
    driver.quit()
    print("Proceso completado.")

In [5]:
if __name__ == "__main__":
    main()

Se cargaron 2353 registros previos del archivo JSON.
Continuando desde la última URL procesada: https://pe.computrabajo.com/empleos-en-lima?p=164
Procesando página: https://pe.computrabajo.com/empleos-en-lima?p=164
No se encontró el pop-up.
Encontradas 20 ofertas en esta página
Oferta ya procesada: Vendedor de Campo con Movilidad LIMA ESTE Turno de 1pm a 10 pm Gana hasta S/2370
Oferta ya procesada: 20 Vendedores o Mercaderista de Campo LIMA ESTE Canal Tradicional GANA S/1900 Comisión Fija
Oferta ya procesada: Impulsadora/Consultora/Productos de belleza/Campaña/Aruma Real Plaza Villa Maria del triunfo
Oferta ya procesada: Impulsadora/Consultora/Productos de belleza/Campaña/Aruma La Rambla Brasil
Oferta ya procesada: Atención al cliente
Oferta ya procesada: Operarios de Producción
Oferta ya procesada: Auxiliar administrativo contable
Oferta ya procesada: Asesor de Retenciones Call Center/ Part Time Tarde/ Sede Independencia/Con o Sin Experiencia
Oferta ya procesada: Asistente de recursos

KeyboardInterrupt: 